https://medium.com/enterprise-rag/an-introduction-to-rag-and-simple-complex-rag-9c3aa9bd017b
https://medium.com/@himanshuit3036/day-3-building-our-first-rag-e5b49b6cb6f6


In [30]:
import dotenv
import os

dotenv.load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [31]:
# Open pdf file
import PyPDF3

book_path = '../data/raw/don_quijote/libro.pdf'

pdf = open(book_path, 'rb')
pdfdoc = PyPDF3.PdfFileReader(pdf)

In [32]:
pdfdoc.numPages

471

In [33]:
page_one= pdfdoc.getPage(0)
print(page_one.extractText())

Miguel de Cervantes
EL INGENIOSO HIDALGO
   DON  QUIJOTE
 DE  LA  MANCHA
No me imprimas, salva un árbol. ¿Para qué malgastar papel?



In [34]:
# Get lists with texts of N pages
pages_in_each_list = 1
group_of_pages = []
for i in range(0, pdfdoc.numPages, pages_in_each_list):
    temp_texts = [page.extractText() for page in pdfdoc.pages[i:i+pages_in_each_list]]
    complete_text = ' '.join(temp_texts)
    group_of_pages.append(complete_text)

In [35]:
group_of_pages[0]

'Miguel de Cervantes\nEL INGENIOSO HIDALGO\n   DON  QUIJOTE\n DE  LA  MANCHA\nNo me imprimas, salva un árbol. ¿Para qué malgastar papel?\n'

In [36]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [37]:
# TODO: Find a way to not use embedding of OpenAI

vectorstore = FAISS.from_texts(texts=group_of_pages, embedding=OpenAIEmbeddings(api_key=api_key))

In [38]:
prompt = """Tu eres un profesor experto en lengua española. Tu tienes que responder preguntas relacionadas con libros. 
Considera el contexto y la pregunta del usuario mostradods abajo.
context = {context}
question = {question}
"""
prompt = ChatPromptTemplate.from_template(prompt)

In [39]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key = api_key)
retriever = vectorstore.as_retriever()

In [40]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [41]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [42]:
rag_chain.invoke("¿Qué le prometió el Quijote a Sancho Panza?")

'Don Quijote le prometió a Sancho Panza la ínsula, es decir, un territorio insular, como recompensa por sus servicios como escudero.'